In [1]:
import os

# Path to check
file_path = r"dynamic files\Main_pages\Recommendations\Learn_page_recommendation"

# Check if the file exists
file_exists = os.path.exists(file_path)
file_exists


True

In [ ]:
# import json

# # Read JSON data from data.json
# with open(r'dynamic files\Main_pages\Learn.json', 'r') as json_file:
#     json_data = json.load(json_file)

# # Prepare the output list
# output_lines = []

# for box in json_data.values():
#     title = box['Title']
#     status = "Completed" if box['Completed'] else "Not Completed"
#     output_lines.append(f'"{title}" : "{status}"')

# # Write to the text file
# with open('exp.txt', 'w') as file:
#     for line in output_lines:
#         file.write(line + '\n')

# print("Data has been written to output.txt")


## recommendation

In [23]:
import os
import ast
import json
from dotenv import load_dotenv
import google.generativeai as genai

load_dotenv()

genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
Learn_list_file_path = r"dynamic files\Main_pages\Learn.json"

def learn_recommendation():
    with open(Learn_list_file_path, 'r') as f:
        data = json.load(f)

    generation_config = {
    "temperature": 0.05,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
    }

    model = genai.GenerativeModel(
    model_name="gemini-1.5-pro",
    generation_config=generation_config,
    )

    chat_session = model.start_chat(
    history=[
        {
        "role": "user",
        "parts": [
            f"""You are provided with a JSON file containing a list of courses. Each course has an \"Image Box\" identifier, a \"Title\", and a \"Completed\" status. Your task is to generate a list of 5 recommended courses based on the following criteria:\n\n1. **Exclusion of Completed Courses:** Only include courses where `\"Completed\": false`.\n2. **Order of Difficulty:** Arrange the recommended courses from easiest to hardest based on their topics.\n3. **Selection When All Courses Are Completed:** If all courses are marked as completed, select any 5 courses that are most suitable.\n4. **Output Format:** The recommendations should be a list of \"Image Box\" identifiers, strictly adhering to the format below.\n\n**Input JSON:**\n```{data}```\n[\"Image Box X\", \"Image Box Y\", \"Image Box Z\", \"Image Box A\", \"Image Box B\"]\n```\n\n**Additional Notes:**\n- Ensure that the list strictly contains only the \"Image Box\" identifiers.\n- Maintain the sequence from the easiest to the most challenging topics.\n- Do not include any courses that have `\"Completed\": true`.\n- If all courses are completed, select any 8 courses that best fit the recommendation criteria.\n- Adhere strictly to the expected output format without deviations.\n""",
        ],
        },
        {
        "role": "model",
        "parts": [
            "```\n[\"Image Box 2\", \"Image Box 3\", \"Image Box 7\", \"Image Box 9\", \"Image Box 11\"]\n``` \n",
        ],
        },
    ]
    )

    response = chat_session.send_message("INSERT_INPUT_HERE")

    temp_text = response.text
    string_data = temp_text.replace('```', '')
    list_response = ast.literal_eval(string_data)

    return list_response

In [25]:
res = learn_recommendation()

In [26]:
res

['Image Box 2', 'Image Box 4', 'Image Box 5', 'Image Box 6', 'Image Box 7']

## Test

In [22]:
import os
import shutil

def delete_all_files_in_directory(directory_path):
    try:
        for filename in os.listdir(directory_path):
            file_path = os.path.join(directory_path, filename)
            if os.path.isfile(file_path):
                os.remove(file_path)
                print(f"Deleted: {file_path}")
            else:
                print(f"Skipped (not a file): {file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

def copy_imagesto_folder_from_list(Image_list_json_path, Images_path, destination_path):
    delete_all_files_in_directory(destination_path)
    os.makedirs(destination_path, exist_ok=True)

    with open(Image_list_json_path, 'r') as file:
        Image_list = json.load(file)

    for image_name in Image_list:
        source_file = os.path.join(Images_path, f"{image_name}.jpg")
        if os.path.isfile(source_file):
            shutil.copy(source_file, destination_path)
            print(f"Copied: {source_file} to {destination_path}")
        else:
            print(f"File does not exist: {source_file}")

# List of images to copy
learn_Image_list_json_path = r"dynamic files\Main_pages\Recommendations\Learn_Recommendation.json"
learn_Images_path = r"Static_Files\Learn_Page\All_Courses"
learn_destination_path = r"dynamic files\Main_pages\Recommendations\Learn_page_recommendation"
copy_imagesto_folder_from_list(learn_Image_list_json_path, learn_Images_path, learn_destination_path)

Copied: Static_Files\Learn_Page\All_Courses\Image Box 2.jpg to TEST
Copied: Static_Files\Learn_Page\All_Courses\Image Box 4.jpg to TEST
Copied: Static_Files\Learn_Page\All_Courses\Image Box 5.jpg to TEST
Copied: Static_Files\Learn_Page\All_Courses\Image Box 6.jpg to TEST
Copied: Static_Files\Learn_Page\All_Courses\Image Box 7.jpg to TEST
